In [ ]:
# in_class_4_str.py
# Professor-style implementation for fetching data, computing returns, and plotting.
# Requires: yfinance, pandas, numpy, matplotlib, seaborn (theme optional)

import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sb  # optional theme

In [ ]:
# Set a clean default theme similar to class materials
sb.set_theme(style="whitegrid", context="talk")

DEFAULT_START = dt.date.isoformat(dt.date.today() - dt.timedelta(365))
DEFAULT_END = dt.date.isoformat(dt.date.today())

In [ ]:
class Stock:
    def __init__(self, symbol: str, start: str = DEFAULT_START, end: str = DEFAULT_END):
        """
        Parameters
        ----------
        symbol : str
            Ticker string, e.g., 'AAPL'
        start : str
            ISO date 'YYYY-MM-DD' (defaults to 1 year ago)
        end : str
            ISO date 'YYYY-MM-DD' (defaults to today)
        """
        if not isinstance(symbol, str):
            raise ValueError("symbol must be a string ticker, e.g., 'AAPL'")
        self.symbol = symbol.upper()
        self.start = start
        self.end = end
        self.data = self.get_data()

    def get_data(self) -> pd.DataFrame:
        """
        Download historical OHLCV data via yfinance and store in a pandas DataFrame.
        - Index set to DatetimeIndex named 'date'
        - Keep a 'close' column (float)
        - Enrich with 'change' and 'instant_return' via calc_returns()
        """
        try:
            import yfinance as yf  # local import
        except Exception as e:
            raise ImportError("Please install yfinance: pip install yfinance") from e

        raw = yf.download(
            self.symbol,
            start=self.start,
            end=self.end,
            interval="1d",
            auto_adjust=False,
            progress=False,
            threads=True,
        )

        if raw is None or raw.empty:
            raise ValueError(f"No data returned for {self.symbol} between {self.start} and {self.end}.")

        data = raw.copy()
        data.index = pd.to_datetime(data.index)
        data.index.name = "date"

        # Standardize the close column
        if "Close" in data.columns:
            data = data[["Close"]].rename(columns={"Close": "close"}).astype(float)
        else:
            # Fallback if column names differ
            close_candidates = [c for c in data.columns if c.lower() == "close"]
            if not close_candidates:
                raise ValueError("Downloaded data does not include a 'Close' column.")
            data = data[[close_candidates[0]]].rename(columns={close_candidates[0]: "close"}).astype(float)

        # Enrich with change & instant_return
        self.calc_returns(data)

        self.data = data
        return data

    def calc_returns(self, df: pd.DataFrame) -> None:
        """
        Helper to add:
        - change: close-to-close dollar change vs. prior day
        - instant_return: daily log return, rounded to 4 decimals
          np.log(close).diff().round(4)
        """
        if "close" not in df.columns:
            raise ValueError("DataFrame must contain a 'close' column before calc_returns().")

        df["change"] = df["close"].diff()
        df["instant_return"] = np.log(df["close"]).diff().round(4)

    def plot_return_dist(self, bins: int = 50) -> None:
        """
        Plot histogram of instantaneous (log) returns.
        """
        if self.data is None or self.data.empty:
            raise ValueError("No data to plot. Instantiate the object or call get_data first.")

        returns = self.data["instant_return"].dropna()
        if returns.empty:
            raise ValueError("instant_return series is empty; cannot plot distribution.")

        plt.figure(figsize=(10, 6))
        plt.hist(returns.values, bins=bins, edgecolor="black", alpha=0.8)
        plt.title(f"{self.symbol} — Daily Instantaneous Return Distribution", pad=14)
        plt.xlabel("Instantaneous Return (log)")
        plt.ylabel("Frequency")
        plt.tight_layout()
        plt.show()

    def plot_performance(self) -> None:
        """
        Plot cumulative performance as percent gain/loss from the first close in the sample.
        """
        if self.data is None or self.data.empty:
            raise ValueError("No data to plot. Instantiate the object or call get_data first.")

        close = self.data["close"].dropna()
        if close.empty:
            raise ValueError("No 'close' prices available for performance plot.")

        base = float(close.iloc[0])
        series = (close / base) - 1.0  # decimal return from baseline

        fig = plt.figure(figsize=(12, 6))
        ax = fig.gca()
        ax.plot(series.index, series.values, linewidth=2)

        ax.set_title(f"{self.symbol} — Performance Since {series.index[0].date().isoformat()}", pad=14)
        ax.set_xlabel("Date")
        ax.set_ylabel("Return (from start)")
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
        ax.grid(True, which="major", alpha=0.3)
        fig.tight_layout()
        plt.show()


def main():
    # Example testing block (uncomment to run as a script):
    # test = Stock(symbol="AAPL")  # Or provide custom range with start=..., end=...
    # print(test.data.head())
    # test.plot_performance()
    # test.plot_return_dist()
    pass


if __name__ == "__main__":
    main()
